### Random Walk Generation
Recall that the stochastic model for asset returns is given by the Geometric Brownian Motion process:
$$ \frac{S_{t+dt}-S_t}{S_t}=\mu dt +\sigma \sqrt{dt} \xi_t $$

When we generate simulated returns, we can usually ignore the decompositino of $ \mu $, since we only care about the net effective drift term without worrying about what the components of it are.

The implementation is very close to the equation above, although it can be optimized in many ways, this is the most direct translation of the equation.
``` Python
import numpy as np

def gbm(
    n_years = 10,
    n_scenarios = 1000,
    mu = 0.07,
    sigma = 0.15,
    steps_per_year = 12,
    s_0 = 100.0
):
    """
    Evolution of Geometric Brownian Motion trajectories, such as for Stock Prices
    :param n_years: The number of years to generate data for
    :param n_paths: The number of scenarios/trajectories
    :param mu: Annualized Drift, e.g., Market Return
    :param sigma: Annualized Volatility
    :param steps_per_year: grranularity of the simulation
    :param s_0: initial value
    :return: a numpy array of n_paths columns and n_years * steps_per_year_rows.
    """
    # Derive per-step model parameters from user specifications
    dt = 1/steps_per_year
    n_steps = int(n_years * steps_per_year)
    xi = np.random.normal(size = (n_steps, n_scenarios))

    # start with just the returns, biased by mu_dt plus 1 since we wil need that later.
    rets = mu * dt + sigma * np.sqrt(dt) * xi
    prices = s_0 * (1 + rets).cumprod()

    return prices
```

In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
def gbm0(
    n_years = 10,
    n_scenarios = 1000,
    mu = 0.07,
    sigma = 0.15,
    steps_per_year = 12,
    s_0 = 100.0
):
    """
    Evolution of Geometric Brownian Motion trajectories, such as for Stock Prices
    :param n_years: The number of years to generate data for
    :param n_paths: The number of scenarios/trajectories
    :param mu: Annualized Drift, e.g., Market Return
    :param sigma: Annualized Volatility
    :param steps_per_year: grranularity of the simulation
    :param s_0: initial value
    :return: a numpy array of n_paths columns and n_years * steps_per_year_rows.
    """
    # Derive per-step model parameters from user specifications
    dt = 1/steps_per_year
    n_steps = int(n_years * steps_per_year)
    xi = np.random.normal(size = (n_steps, n_scenarios))

    # start with just the returns, biased by mu_dt plus 1 since we wil need that later.
    rets = mu * dt + sigma * np.sqrt(dt) * xi
    # convert to a DataFrame
    rets = pd.DataFrame(rets)
    # covert to prices
    prices = s_0 * (1 + rets).cumprod()

    return prices

In [8]:
gbm0(n_years = 5, n_scenarios=1000, mu= 0.07).plot(figsize = (12, 5), legend = False)

AttributeError: 'numpy.ndarray' object has no attribute 'plot'